In [ ]:
!pip install folium

In [ ]:
pip install geopandas

In [ ]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import geopandas as gpd
import folium
from shapely.geometry import Point, Polygon, LineString
import matplotlib as mpl
import pickle

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
Bike_Station=pd.read_csv('02.자전거스테이션.csv')
Administrative_boundary = gpd.read_file('08.행정경계(시군구).geojson')
space_plan=gpd.read_file('10.도시계획(공간시설).geojson')
culphy_plan=gpd.read_file('11.도시계획(공공문화제육시설).geojson')
traffic_plan=gpd.read_file('12.도시계획(교통시설).geojson')
wetland_cons=gpd.read_file('13.용도지역지구(습지보호지역).geojson')
build_road=gpd.read_file('15.도로명주소_건물.geojson')
build_road_plus = pd.read_pickle('도로명주소_건물')
road_road=gpd.read_file('16.도로명주소_도로.geojson') 
build_dist=gpd.read_file('17.일반건물 분포도(100M X 100M).geojson')
fes_space=gpd.read_file('18.행사장_공간정보.csv')
subway_station=gpd.read_file('19.전철역_공간정보.csv')
bus_station=gpd.read_file('20.고양시 버스정류소.csv')
parking_info=gpd.read_file('22.주차장정보.csv')
theat_muse=gpd.read_file('26.고양시 공연장 박물관 정보.csv')
physi_curr=gpd.read_file('27.고양시 체육시설 현황 정보.csv')

## 기존 계획 데이터

In [ ]:
space_plan['분류'] = '도시계획(공간시설)'
space_plan['UQT_m']=space_plan['MNUM'].apply(lambda x : x[20:-7]) # # 용도지역 지구 구분
space_plan['라벨'] = np.nan
space_plan

,MNUM,ALIAS,REMARK,geometry,분류,UQT_m,라벨
0,39400004128020200102UQT3300001000,None,None,"MULTIPOLYGON (((126.79244 37.67061, 126.79245 ...",도시계획(공간시설),UQT330,NaN
1,39400004128020200102UQT3300002000,None,None,"MULTIPOLYGON (((126.79125 37.66747, 126.79126 ...",도시계획(공간시설),UQT330,NaN
2,39400004128020200102UQT2100001000,None,None,"MULTIPOLYGON (((126.79216 37.67002, 126.79221 ...",도시계획(공간시설),UQT210,NaN
3,39400004128020200102UQT3100001000,None,None,"MULTIPOLYGON (((126.79275 37.66799, 126.79301 ...",도시계획(공간시설),UQT310,NaN
4,39400004128020200102UQT3100002000,None,None,"MULTIPOLYGON (((126.79139 37.66740, 126.79154 ...",도시계획(공간시설),UQT310,NaN
...,...,...,...,...,...,...,...
1089,16130004128020130925UQT3200087000,None,None,"MULTIPOLYGON (((126.87392 37.64883, 126.87388 ...",도시계획(공간시설),UQT320,NaN
1090,16130004128020150203UQT2200005000,None,None,"MULTIPOLYGON (((126.88345 37.66780, 126.88346 ...",도시계획(공간시설),UQT220,NaN
1091,16130004128020150203UQT2200003000,None,None,"MULTIPOLYGON (((126.88881 37.64833, 126.88854 ...",도시계획(공간시설),UQT220,NaN
1092,16130004128020150203UQT2200002000,None,None,"MULTIPOLYGON (((126.88267 37.64497, 126.88218 ...",도시계획(공간시설),UQT220,NaN


In [ ]:
culphy_plan['분류'] = '도시계획(공공문화체육시설)'
culphy_plan['라벨'] = np.nan
culphy_plan

,DGM_NM,DGM_AR,DGM_LT,geometry,분류,라벨
0,기타 사회복지시설,2627,211,"MULTIPOLYGON (((126.86736 37.62940, 126.86761 ...",도시계획(공공문화체육시설),NaN
1,초등학교,13070,461,"MULTIPOLYGON (((126.90542 37.71515, 126.90619 ...",도시계획(공공문화체육시설),NaN
2,기타 사회복지시설,560,95,"MULTIPOLYGON (((126.78826 37.67892, 126.78836 ...",도시계획(공공문화체육시설),NaN
3,초등학교,12000,444,"MULTIPOLYGON (((126.81611 37.68054, 126.81659 ...",도시계획(공공문화체육시설),NaN
4,유치원,3000,219,"MULTIPOLYGON (((126.81012 37.68488, 126.81062 ...",도시계획(공공문화체육시설),NaN
...,...,...,...,...,...,...
403,초등학교,6313,361,"MULTIPOLYGON (((126.78123 37.68526, 126.78126 ...",도시계획(공공문화체육시설),NaN
404,골프장,295598,3041,"MULTIPOLYGON (((126.85537 37.63687, 126.85545 ...",도시계획(공공문화체육시설),NaN
405,기타 사회복지시설,1063,137,"MULTIPOLYGON (((126.82721 37.70204, 126.82740 ...",도시계획(공공문화체육시설),NaN
406,기타 사회복지시설,935,150,"MULTIPOLYGON (((126.90365 37.70301, 126.90365 ...",도시계획(공공문화체육시설),NaN


In [ ]:
traffic_plan['분류'] = '도시계획(교통시설)'
traffic_plan['라벨'] = np.nan
traffic_plan

,DGM_NM,DGM_AR,DGM_LT,geometry,분류,라벨
0,노외주차장,1930,269,"MULTIPOLYGON (((126.89527 37.70315, 126.89514 ...",도시계획(교통시설),NaN
1,노외주차장,1006,130,"MULTIPOLYGON (((126.79379 37.65890, 126.79391 ...",도시계획(교통시설),NaN
2,노외주차장,1111,134,"MULTIPOLYGON (((126.91072 37.71903, 126.91076 ...",도시계획(교통시설),NaN
3,노외주차장,631,104,"MULTIPOLYGON (((126.89455 37.62056, 126.89480 ...",도시계획(교통시설),NaN
4,노외주차장,987,132,"MULTIPOLYGON (((126.87241 37.59324, 126.87251 ...",도시계획(교통시설),NaN
...,...,...,...,...,...,...
318,노외주차장,2057,188,"MULTIPOLYGON (((126.80513 37.62288, 126.80527 ...",도시계획(교통시설),NaN
319,노외주차장,420,93,"MULTIPOLYGON (((126.85073 37.64267, 126.85075 ...",도시계획(교통시설),NaN
320,노외주차장,982,150,"MULTIPOLYGON (((126.86762 37.61254, 126.86787 ...",도시계획(교통시설),NaN
321,노외주차장,621,106,"MULTIPOLYGON (((126.81503 37.60231, 126.81503 ...",도시계획(교통시설),NaN


In [ ]:
# 좌표계 확인
print('space_plan',space_plan.crs)
print('culphy_plan',culphy_plan.crs)
print('traffic_plan',traffic_plan.crs)

space_plan epsg:4326
culphy_plan epsg:4326
traffic_plan epsg:4326


## 기존 실제 데이터

In [ ]:
build_road_plus # build_road_plus.ipynb 파일에서 만든 

,BDTYP_CD,BULD_NM,BULD_NM_DC,EMD_CD,GRO_FLO_CO,SIG_CD,UND_FLO_CO,geometry,geometry_Point,코드값 의미,BDTYP_CD_코드값의미
0,01003,None,None,105,4.0,41281,0.0,"MULTIPOLYGON (((126.87256 37.62531, 126.87259 ...",POINT (126.87260 37.62525),다가구주택,01003_다가구주택
1,01003,None,None,106,3.0,41285,0.0,"MULTIPOLYGON (((126.78974 37.64674, 126.78980 ...",POINT (126.78972 37.64668),다가구주택,01003_다가구주택
2,01003,None,None,106,3.0,41281,0.0,"MULTIPOLYGON (((126.85353 37.64523, 126.85360 ...",POINT (126.85361 37.64514),다가구주택,01003_다가구주택
3,01003,None,None,103,3.0,41285,0.0,"MULTIPOLYGON (((126.77427 37.67318, 126.77433 ...",POINT (126.77435 37.67319),다가구주택,01003_다가구주택
4,01003,None,B동,107,4.0,41285,0.0,"MULTIPOLYGON (((126.79842 37.67501, 126.79854 ...",POINT (126.79848 37.67495),다가구주택,01003_다가구주택
...,...,...,...,...,...,...,...,...,...,...,...
79997,27999,None,None,104,5.0,41287,1.0,"MULTIPOLYGON (((126.73415 37.66356, 126.73435 ...",POINT (126.73438 37.66325),기타관광휴게시설,27999_기타관광휴게시설
79998,90002,None,A30,123,0.0,41281,0.0,"MULTIPOLYGON (((126.83330 37.63768, 126.83327 ...",POINT (126.83328 37.63770),무,90002_무
79999,90002,None,None,123,0.0,41281,0.0,"MULTIPOLYGON (((126.83208 37.63485, 126.83211 ...",POINT (126.83210 37.63484),무,90002_무
80000,90002,None,C62,105,0.0,41287,0.0,"MULTIPOLYGON (((126.75660 37.69236, 126.75659 ...",POINT (126.75661 37.69237),무,90002_무


In [ ]:
print('build_road_plus',build_road_plus.crs)

build_road_plus epsg:4326


In [ ]:
# point geometry 생성
def to_float(df):
    df['X'] = df['X'].astype('float')
    df['Y'] = df['Y'].astype('float')
    df['geometry'] = df.apply(lambda row : Point([row['X'], row['Y']]),axis=1)
    df = gpd.GeoDataFrame(df, geometry='geometry')
    return df

In [ ]:
fes_space['분류'] = '행사장_공간정보'
fes_space.columns = ['Name','Y', 'X', "geometry", '분류']
fes_space = fes_space[['Name','X', 'Y', "geometry", '분류']]
fes_space = to_float(fes_space)
fes_space

,Name,X,Y,geometry,분류
0,KINTEX,126.746184,37.669028,POINT (126.74618 37.66903),행사장_공간정보
1,꽃 박람회,126.767543,37.655155,POINT (126.76754 37.65516),행사장_공간정보


In [ ]:
subway_station['분류'] = '전철역_공간정보'
subway_station['Name']=subway_station[['station_nm','rail_nm']].apply(lambda x: '_'.join(x), axis=1)
subway_station = subway_station.loc[:,['Name','road_nm_addr','lot_num_addr','X','Y','geometry','분류']]
subway_station = to_float(subway_station)
subway_station

,Name,road_nm_addr,lot_num_addr,X,Y,geometry,분류
0,대곡_3호선,경기도 고양시 덕양구 대주로107번길 71-81,경기도 고양시 덕양구 대장동 420-1,126.809614,37.632306,POINT (126.80961 37.63231),전철역_공간정보
1,대화_3호선,경기도 고양시 일산서구 중앙로 지하 1569,경기도 고양시 일산서구 대화동 2221,126.747206,37.675846,POINT (126.74721 37.67585),전철역_공간정보
2,일산_경의중앙,경기도 고양시 일산서구 경의로 672,경기도 고양시 일산서구 일산동 655-691,126.770067,37.682007,POINT (126.77007 37.68201),전철역_공간정보
3,백석_3호선,경기도 고양시 일산동구 중앙로 지하 1042,경기도 고양시 일산동구 백석동 1538,126.788148,37.642953,POINT (126.78815 37.64295),전철역_공간정보
4,대곡_경의중앙,경기도 고양시 덕양구 대주로 107번길 71-81,경기도 고양시 덕양구 대장동 420-1,126.811047,37.631615,POINT (126.81105 37.63161),전철역_공간정보
5,강매_경의중앙,경기도 고양시 덕양구 소원로 202,경기도 고양시 덕양구 행신동 1115-1,126.844436,37.612290,POINT (126.84444 37.61229),전철역_공간정보
6,백마_경의중앙,경기도 고양시 일산동구 경의로 318,경기도 고양시 일산동구 백석동 92,126.794067,37.658431,POINT (126.79407 37.65843),전철역_공간정보
7,능곡_경의중앙,경기도 고양시 덕양구 토당로 35,경기도 고양시 덕양구 토당동 454-3,126.820838,37.618765,POINT (126.82084 37.61877),전철역_공간정보
8,풍산_경의중앙,경기도 고양시 일산동구 경의로 486,경기도 고양시 일산동구 풍동 1042,126.786528,37.671815,POINT (126.78653 37.67182),전철역_공간정보
9,행신_경의중앙,경기도 고양시 덕양구 소원로 114,경기도 고양시 덕양구 행신동 812,126.834117,37.612170,POINT (126.83412 37.61217),전철역_공간정보


In [ ]:
parking_info['분류'] = '주차장정보'
parking_info['Name'] = parking_info[['parking_nm','space']].apply(lambda x: '_'.join(x), axis=1)
parking_info = parking_info.loc[:,['Name','road_nm_addr', 'lot_num_addr', 'X','Y','geometry','분류']]
parking_info = to_float(parking_info)
parking_info

,Name,road_nm_addr,lot_num_addr,X,Y,geometry,분류
0,고양시청 부설주차장_140,경기도 고양시 덕양구 고양시청로10,경기도 고양시 덕양구 주교동 600,126.832072,37.658042,POINT (126.83207 37.65804),주차장정보
1,고양제1_94,경기도 고양시 덕양구 벽제관로 8,경기도 고양시 덕양구 고양동 195-2,126.903861,37.703783,POINT (126.90386 37.70378),주차장정보
2,고양제2_31,경기도 고양시 덕양구 혜음로 103,경기도 고양시 덕양구 고양동 444-1,126.905347,37.710213,POINT (126.90535 37.71021),주차장정보
3,노상 1권역_97,경기도 고양시 덕양구 고양시청로 10,경기도 고양시 덕양구 주교동 600,126.831960,37.658414,POINT (126.83196 37.65841),주차장정보
4,노상 3권역_123,경기도 고양시 덕양구 은빛로 39,경기도 고양시 덕양구 화정동 909,126.832173,37.638096,POINT (126.83217 37.63810),주차장정보
...,...,...,...,...,...,...,...
65,호수공원 부설주차장 2_512,경기도 고양시 일산동구 호수로 595,경기도 고양시 일산동구 장항동 906,126.763193,37.661923,POINT (126.76319 37.66192),주차장정보
66,호수공원 부설주차장 3_54,경기도 고양시 일산동구 호수로 595,경기도 고양시 일산동구 장항동 906,126.769069,37.654901,POINT (126.76907 37.65490),주차장정보
67,호수공원 부설주차장 4_125,경기도 고양시 일산동구 호수로 595,경기도 고양시 일산동구 장항동 906,126.770942,37.651669,POINT (126.77094 37.65167),주차장정보
68,화정환승_253,경기도 고양시 덕양구 화정로 지하 63,경기도 고양시 덕양구 화정동 963,126.833046,37.634951,POINT (126.83305 37.63495),주차장정보


In [ ]:
theat_muse['분류'] = '고양시 공연장 박물관 정보'
theat_muse['Name'] =theat_muse[['place_nm','servicetype']].apply(lambda x: '_'.join(x), axis=1)
theat_muse = theat_muse.loc[:,['Name', 'X','Y','geometry','분류']]
theat_muse = to_float(theat_muse)
theat_muse

,Name,X,Y,geometry,분류
0,활주로7080 공연장_공연장,126.767487,37.663502,POINT (126.76749 37.66350),고양시 공연장 박물관 정보
1,행주아트홀_공연장,126.822885,37.602563,POINT (126.82288 37.60256),고양시 공연장 박물관 정보
2,올리브_공연장,126.767880,37.663102,POINT (126.76788 37.66310),고양시 공연장 박물관 정보
3,3 NEW THEATRE_공연장,126.725248,37.690864,POINT (126.72525 37.69086),고양시 공연장 박물관 정보
4,터 시어터_공연장,126.782173,37.659433,POINT (126.78217 37.65943),고양시 공연장 박물관 정보
5,벧엘뮤직홀_공연장,126.772393,37.658009,POINT (126.77239 37.65801),고양시 공연장 박물관 정보
6,우경아트홀(k뮤직아카데미)_공연장,126.831048,37.631420,POINT (126.83105 37.63142),고양시 공연장 박물관 정보
7,레미어린이극장_공연장,126.753790,37.667195,POINT (126.75379 37.66719),고양시 공연장 박물관 정보
8,나가수공연장_공연장,126.766667,37.663524,POINT (126.76667 37.66352),고양시 공연장 박물관 정보
9,노루목야외극장_공연장,126.774647,37.661522,POINT (126.77465 37.66152),고양시 공연장 박물관 정보


In [ ]:
physi_curr['분류'] = '고양시 체육시설 현황 정보'
physi_curr['Name'] =physi_curr[['FAC_NAME','GBN']].apply(lambda x: '_'.join(x), axis=1)
physi_curr = physi_curr.loc[:,['Name','X','Y','geometry','분류']]
physi_curr = to_float(physi_curr)
physi_curr

,Name,X,Y,geometry,분류
0,"고양종합운동장(주,보조)_육상",126.742594,37.675787,POINT (126.74259 37.67579),고양시 체육시설 현황 정보
1,어울림 별무리 운동장_육상,126.834639,37.649847,POINT (126.83464 37.64985),고양시 체육시설 현황 정보
2,"고양종합운동장 축구장(주,보조)_축구장",126.742594,37.675787,POINT (126.74259 37.67579),고양시 체육시설 현황 정보
3,백석공원 축구장_축구장,126.794876,37.645546,POINT (126.79488 37.64555),고양시 체육시설 현황 정보
4,중산중앙 근린공원축구장_축구장,126.778620,37.691654,POINT (126.77862 37.69165),고양시 체육시설 현황 정보
...,...,...,...,...,...
172,고양백석체육센터 GX장_다목적시설,126.796418,37.641976,POINT (126.79642 37.64198),고양시 체육시설 현황 정보
173,원능하수처리장축구장_축구장,126.794780,37.624858,POINT (126.79478 37.62486),고양시 체육시설 현황 정보
174,창릉동운동장_축구장,126.893888,37.641831,POINT (126.89389 37.64183),고양시 체육시설 현황 정보
175,일산3동 게이트볼장_게이트볼장,126.763313,37.680025,POINT (126.76331 37.68003),고양시 체육시설 현황 정보


In [ ]:
# 계획 데이터와 같은 좌표계 사용
fes_space.set_crs('epsg:4326')
subway_station.set_crs('epsg:4326')
parking_info.set_crs('epsg:4326')
theat_muse.set_crs('epsg:4326')
physi_curr.set_crs('epsg:4326')

,Name,X,Y,geometry,분류
0,"고양종합운동장(주,보조)_육상",126.742594,37.675787,POINT (126.74259 37.67579),고양시 체육시설 현황 정보
1,어울림 별무리 운동장_육상,126.834639,37.649847,POINT (126.83464 37.64985),고양시 체육시설 현황 정보
2,"고양종합운동장 축구장(주,보조)_축구장",126.742594,37.675787,POINT (126.74259 37.67579),고양시 체육시설 현황 정보
3,백석공원 축구장_축구장,126.794876,37.645546,POINT (126.79488 37.64555),고양시 체육시설 현황 정보
4,중산중앙 근린공원축구장_축구장,126.778620,37.691654,POINT (126.77862 37.69165),고양시 체육시설 현황 정보
...,...,...,...,...,...
172,고양백석체육센터 GX장_다목적시설,126.796418,37.641976,POINT (126.79642 37.64198),고양시 체육시설 현황 정보
173,원능하수처리장축구장_축구장,126.794780,37.624858,POINT (126.79478 37.62486),고양시 체육시설 현황 정보
174,창릉동운동장_축구장,126.893888,37.641831,POINT (126.89389 37.64183),고양시 체육시설 현황 정보
175,일산3동 게이트볼장_게이트볼장,126.763313,37.680025,POINT (126.76331 37.68003),고양시 체육시설 현황 정보


# 데이터 확인(실제 데이터)
- build_road_plus
- physi_curr
- fes_space
- subway_station
- parking_info
- theat_muse

### build_road_plus 데이터 확인
- space_plan
- culphy_plan
- traffic_plan

In [ ]:
# space_plan
for i in build_road_plus.index:
    for j in space_plan.index:
        if build_road_plus.geometry_Point[i].within(space_plan.geometry[j]):
            print('{}는 {}에 속해있다.'.format(build_road_plus.BDTYP_CD_코드값의미[i],space_plan.MNUM[j]))

01003_다가구주택는 16130004128020130925UQT3200073000에 속해있다.
01003_다가구주택는 39400004128020180033UQT2200003000에 속해있다.
01003_다가구주택는 39400004128020180033UQT2200003000에 속해있다.
01003_다가구주택는 39400004128020180033UQT2100002000에 속해있다.
01003_다가구주택는 39400004128020180033UQT2100002000에 속해있다.
01003_다가구주택는 39400004128020180033UQT2100002000에 속해있다.
01003_다가구주택는 39400004128020180033UQT2100002000에 속해있다.
01003_다가구주택는 16130004128020151010UQT3100013000에 속해있다.
01003_다가구주택는 41280004128020100246UQT5000026000에 속해있다.
01003_다가구주택는 16130004128020190723UQT1100001000에 속해있다.
01003_다가구주택는 16130004128020151010UQT3100013000에 속해있다.
02003_다세대주택는 39400004128020160255UQT3200001000에 속해있다.
03999_기타제1종근린생활시설는 39400004128020170227UQT2200047000에 속해있다.
03999_기타제1종근린생활시설는 39400004128020170227UQT2200047000에 속해있다.
03999_기타제1종근린생활시설는 16130004128020190723UQT1100003000에 속해있다.
03999_기타제1종근린생활시설는 61400004128120065203UQT3100107000에 속해있다.
03999_기타제1종근린생활시설는 16130004128020190826UQT2200008000에 속해있다.
03999_기타제1종근린생활시설는 16130004128020190826UQT2200008000

In [ ]:
# space_plan
for i in build_road_plus.index:
    for j in space_plan.index:
        if build_road_plus.geometry_Point[i].within(space_plan.geometry[j]):
            space_plan['라벨'][j] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
  # culphy_plan
for i in build_road_plus.index:
    for j in culphy_plan.index:
        if build_road_plus.geometry_Point[i].within(culphy_plan.geometry[j]):
            print('{}는 {}에 속해있다.'.format(build_road_plus.BDTYP_CD_코드값의미[i],culphy_plan.DGM_AR[j]))

01003_다가구주택는 2487에 속해있다.
01003_다가구주택는 2487에 속해있다.
01003_다가구주택는 9853에 속해있다.
01003_다가구주택는 9853에 속해있다.
01003_다가구주택는 9853에 속해있다.
01003_다가구주택는 742에 속해있다.
02003_다세대주택는 2939에 속해있다.
02003_다세대주택는 2939에 속해있다.
02003_다세대주택는 2487에 속해있다.
02003_다세대주택는 2487에 속해있다.
02003_다세대주택는 2487에 속해있다.
03999_기타제1종근린생활시설는 2487에 속해있다.
03999_기타제1종근린생활시설는 498450에 속해있다.
03999_기타제1종근린생활시설는 2487에 속해있다.
03999_기타제1종근린생활시설는 1610에 속해있다.
03999_기타제1종근린생활시설는 1610에 속해있다.
03999_기타제1종근린생활시설는 1456에 속해있다.
01001_단독주택는 374040에 속해있다.
01001_단독주택는 374040에 속해있다.
01001_단독주택는 374040에 속해있다.
01001_단독주택는 374040에 속해있다.
01001_단독주택는 374040에 속해있다.
01001_단독주택는 18204에 속해있다.
01001_단독주택는 71584에 속해있다.
01001_단독주택는 71584에 속해있다.
01001_단독주택는 71584에 속해있다.
01001_단독주택는 71584에 속해있다.
01001_단독주택는 71584에 속해있다.
01001_단독주택는 71584에 속해있다.
01001_단독주택는 31959에 속해있다.
01001_단독주택는 13758에 속해있다.
01001_단독주택는 71584에 속해있다.
01001_단독주택는 71584에 속해있다.
01001_단독주택는 6294에 속해있다.
01001_단독주택는 472에 속해있다.
01001_단독주택는 266130에 속해있다.
01001_단독주택는 117857에 속해있다.
01001_단독주택는 928에 속해있다.
01001_단독주택는

In [ ]:
  # culphy_plan
for i in build_road_plus.index:
    for j in culphy_plan.index:
        if build_road_plus.geometry_Point[i].within(culphy_plan.geometry[j]):
            culphy_plan['라벨'][j] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
# traffic_plan
for i in build_road_plus.index:
    for j in traffic_plan.index:
        if build_road_plus.geometry_Point[i].within(traffic_plan.geometry[j]):
            print('{}는 {}에 속해있다.'.format(build_road_plus.BDTYP_CD_코드값의미[i],traffic_plan.DGM_AR[j]))

01003_다가구주택는 1632에 속해있다.
01003_다가구주택는 1632에 속해있다.
01003_다가구주택는 1632에 속해있다.
02003_다세대주택는 549에 속해있다.
02003_다세대주택는 1390에 속해있다.
02003_다세대주택는 587에 속해있다.
03999_기타제1종근린생활시설는 999567에 속해있다.
03999_기타제1종근린생활시설는 55413에 속해있다.
03999_기타제1종근린생활시설는 55415에 속해있다.
03999_기타제1종근린생활시설는 1699에 속해있다.
03999_기타제1종근린생활시설는 1018에 속해있다.
03999_기타제1종근린생활시설는 1337에 속해있다.
03999_기타제1종근린생활시설는 3164에 속해있다.
01001_단독주택는 18272에 속해있다.
01001_단독주택는 263071에 속해있다.
01001_단독주택는 263071에 속해있다.
01001_단독주택는 19451에 속해있다.
01001_단독주택는 16310에 속해있다.
01001_단독주택는 573에 속해있다.
01001_단독주택는 7255에 속해있다.
01001_단독주택는 263071에 속해있다.
01001_단독주택는 263071에 속해있다.
01001_단독주택는 198806에 속해있다.
01001_단독주택는 21479에 속해있다.
01001_단독주택는 21479에 속해있다.
01001_단독주택는 21479에 속해있다.
01001_단독주택는 21479에 속해있다.
01001_단독주택는 789에 속해있다.
01001_단독주택는 263071에 속해있다.
01001_단독주택는 999567에 속해있다.
01001_단독주택는 999567에 속해있다.
01001_단독주택는 8950에 속해있다.
01001_단독주택는 8950에 속해있다.
01001_단독주택는 263071에 속해있다.
01001_단독주택는 263071에 속해있다.
01001_단독주택는 263071에 속해있다.
01001_단독주택는 263071에 속해있다.
01001_단독주택는 999567에 속해있다.


In [ ]:
# traffic_plan
for i in build_road_plus.index:
    for j in traffic_plan.index:
        if build_road_plus.geometry_Point[i].within(traffic_plan.geometry[j]):
            traffic_plan['라벨'][j] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


### physi_curr 데이터 확인
- space_plan
- culphy_plan
- traffic_plan

In [ ]:
# space_plan
for i in physi_curr.index:
    for j in space_plan.index:
        if physi_curr.geometry[i].within(space_plan.geometry[j]):
            print('{}는 {}에 속해있다.'.format(physi_curr.Name[i],space_plan.MNUM[j]))

백석공원 축구장_축구장는 41280004128520080172UQT2200001000에 속해있다.
중산중앙 근린공원축구장_축구장는 41280004128020100223UQT2500022000에 속해있다.
대화레포츠공원축구장_축구장는 39400004128720160227UQT2500159000에 속해있다.
강선공원축구장_축구장는 41280004128720080172UQT2200011000에 속해있다.
성저공원축구장_축구장는 41280004128720080172UQT2200013000에 속해있다.
정발산공원축구장_축구장는 41280004128520080172UQT2200008000에 속해있다.
강촌공원축구장_축구장는 41280004128520080172UQT2200006000에 속해있다.
대화배드민턴장_배드민턴장는 39400004128720160227UQT2500159000에 속해있다.
백석배드민턴장_배드민턴장는 41280004128520080172UQT2200001000에 속해있다.
성라1배드민턴장_배드민턴장는 39400004128120170250UQT2200042000에 속해있다.
성라2배드민턴장_배드민턴장는 39400004128120170250UQT2200042000에 속해있다.
정발산배드민턴장_배드민턴장는 41280004128520080172UQT2200008000에 속해있다.
성사다목적체육관_배드민턴장는 39400004128020200175UQT2500045000에 속해있다.
중산배드민턴장_배드민턴장는 41280004128020100223UQT2500022000에 속해있다.
마상배드민턴장_배드민턴장는 41280004128120080172UQT2200044000에 속해있다.
행신배드민턴장_배드민턴장는 39400004128120150057UQT2200024000에 속해있다.
탄현배드민턴장_배드민턴장는 41280004128720080172UQT2000020700에 속해있다.
탄현배드민턴장_배드민턴장는 39400004128020200175UQT2200020000

In [ ]:
for i in physi_curr.index:
    for j in space_plan.index:
        if physi_curr.geometry[i].within(space_plan.geometry[j]):
            space_plan['라벨'][j] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
# culphy_plan
for i in physi_curr.index:
    for j in culphy_plan.index:
        if physi_curr.geometry[i].within(culphy_plan.geometry[j]):
            print('{}는 {}에 속해있다.'.format(physi_curr.Name[i],culphy_plan.DGM_AR[j]))

고양종합운동장(주,보조)_육상는 298996에 속해있다.
고양종합운동장 축구장(주,보조)_축구장는 298996에 속해있다.
지영동축구장_축구장는 46847에 속해있다.
고양동축구장_축구장는 14749에 속해있다.
고양체육관_수영장는 298996에 속해있다.
성저테니스장_테니스장는 298996에 속해있다.
지영동체육공원게이트볼장_게이트볼장는 46847에 속해있다.
행신동게이트볼장_게이트볼장는 1577에 속해있다.
성저파크골프장_파크골프장는 298996에 속해있다.
지영동체육공원_족구장는 46847에 속해있다.
고양국가대표야구훈련장_야구장는 298996에 속해있다.
킨텍스 야구장_야구장는 27406에 속해있다.
대화동 실내야구연습장_야구장는 298996에 속해있다.
고양인공암벽등반장_인공암벽장는 13224에 속해있다.
고양체육관_농구장는 298996에 속해있다.
지영동국궁장_궁도장는 46847에 속해있다.
장미란체육관 탁구장_탁구장는 3299에 속해있다.
장미란체육관 탁구장_스쿼시는 3299에 속해있다.
고양체육관_스쿼시는 298996에 속해있다.
고양체육관_골프연습장는 298996에 속해있다.
고양체육관_헬스장는 298996에 속해있다.
풋살구장_풋살구장는 11664에 속해있다.
게이트볼장_게이트볼장는 11664에 속해있다.
인라인스케이트장_인라인스케이트장는 11664에 속해있다.
원능하수처리장축구장_축구장는 9522에 속해있다.
일산3동 게이트볼장_게이트볼장는 900에 속해있다.


In [ ]:
# culphy_plan
for i in physi_curr.index:
    for j in culphy_plan.index:
        if physi_curr.geometry[i].within(culphy_plan.geometry[j]):
            culphy_plan['라벨'][j] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
# traffic_plan
for i in physi_curr.index:
    for j in traffic_plan.index:
        if physi_curr.geometry[i].within(traffic_plan.geometry[j]):
            print('{}는 {}에 속해있다.'.format(physi_curr.Name[i],traffic_plan.DGM_AR[j]))

충장축구장_축구장는 1223180에 속해있다.
충장테니스장_테니스장는 1223180에 속해있다.
게이트볼장_게이트볼장는 1223180에 속해있다.
족구장_족구장는 1223180에 속해있다.
풋살구장_풋살구장는 1223180에 속해있다.


In [ ]:
# traffic_plan
for i in physi_curr.index:
    for j in traffic_plan.index:
        if physi_curr.geometry[i].within(traffic_plan.geometry[j]):
            traffic_plan['라벨'][j] = 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


### fes_space 데이터 확인
- space_plan
- culphy_plan
- traffic_plan

In [ ]:
# space_plan
for i in fes_space.index:
    for j in space_plan.index:
        if fes_space.geometry[i].within(space_plan.geometry[j]):
            print('{}는 {}에 속해있다.'.format(fes_space.Name[i],space_plan.MNUM[j]))

꽃 박람회는 39400004128520160227UQT2200016000에 속해있다.


In [ ]:
# space_plan
for i in fes_space.index:
    for j in space_plan.index:
        if fes_space.geometry[i].within(space_plan.geometry[j]):
            space_plan['라벨'][j] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
# culphy_plan
for i in fes_space.index:
    for j in culphy_plan.index:
        if fes_space.geometry[i].within(culphy_plan.geometry[j]):
            print('{}는 {}에 속해있다.'.format(fes_space.Name[i],culphy_plan.DGM_AR[j]))

In [ ]:
# traffic_plan
for i in fes_space.index:
    for j in traffic_plan.index:
        if fes_space.geometry[i].within(traffic_plan.geometry[j]):
            print('{}는 {}에 속해있다.'.format(fes_space.Name[i],traffic_plan.DGM_AR[j]))

### subway_station 데이터 확인
- space_plan
- culphy_plan
- traffic_plan

In [ ]:
# space_plan
for i in subway_station.index:
    for j in space_plan.index:
        if subway_station.geometry[i].within(space_plan.geometry[j]):
            print('{}는 {}에 속해있다.'.format(subway_station.Name[i],space_plan.MNUM[j]))

대화_3호선는 41280004128720080172UQT1000009000에 속해있다.
백석_3호선는 15000004128519950097UQT1000012000에 속해있다.


In [ ]:
# space_plan
for i in subway_station.index:
    for j in space_plan.index:
        if subway_station.geometry[i].within(space_plan.geometry[j]):
            space_plan['라벨'][j] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
# culphy_plan
for i in subway_station.index:
    for j in culphy_plan.index:
        if subway_station.geometry[i].within(culphy_plan.geometry[j]):
            print('{}는 {}에 속해있다.'.format(subway_station.Name[i],culphy_plan.DGM_AR[j]))

In [ ]:
# traffic_plan
for i in subway_station.index:
    for j in traffic_plan.index:
        if subway_station.geometry[i].within(traffic_plan.geometry[j]):
            print('{}는 {}에 속해있다.'.format(subway_station.Name[i],traffic_plan.DGM_AR[j]))

대곡_3호선는 999567에 속해있다.
일산_경의중앙는 999567에 속해있다.
대곡_경의중앙는 265230에 속해있다.
강매_경의중앙는 999567에 속해있다.
강매_경의중앙는 7724에 속해있다.
백마_경의중앙는 999567에 속해있다.
능곡_경의중앙는 999567에 속해있다.
풍산_경의중앙는 999567에 속해있다.
행신_경의중앙는 999567에 속해있다.
화전(한국항공대)_경의중앙는 999567에 속해있다.
탄현_경의중앙는 999567에 속해있다.
주엽_3호선는 75725에 속해있다.
정발산_3호선는 75725에 속해있다.
마두_3호선는 75725에 속해있다.
화정_3호선는 265230에 속해있다.
곡산_경의중앙는 999567에 속해있다.
원흥_3호선는 18272에 속해있다.
원당_3호선는 265230에 속해있다.
삼송_3호선는 8926에 속해있다.
지축역_3호선는 265230에 속해있다.
지축역_3호선는 30173에 속해있다.


In [ ]:
# traffic_plan
for i in subway_station.index:
    for j in traffic_plan.index:
        if subway_station.geometry[i].within(traffic_plan.geometry[j]):
           traffic_plan['라벨'][j] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


### parking_info 데이터 확인
- space_plan
- culphy_plan
- traffic_plan

In [ ]:
# space_plan
for i in parking_info.index:
    for j in space_plan.index:
        if parking_info.geometry[i].within(space_plan.geometry[j]):
            print('{}는 {}에 속해있다.'.format(parking_info.Name[i],space_plan.MNUM[j]))

중산공원 부설주차장_103는 41280004128020100223UQT2500022000에 속해있다.
탄현환승_233는 39400004128020200175UQT3100007000에 속해있다.
행주산성 부설주차장 상_83는 39400004128020170058UQT2000050000에 속해있다.
행주산성 부설주차장 하_165는 39400004128020170058UQT2000050000에 속해있다.
호수공원 부설주차장 1_248는 39400004128520160227UQT2200016000에 속해있다.
호수공원 부설주차장 2_512는 39400004128520160227UQT2200016000에 속해있다.
호수공원 부설주차장 3_54는 39400004128520160227UQT2200016000에 속해있다.
호수공원 부설주차장 4_125는 41280004128520080172UQT2200017000에 속해있다.
화정환승_253는 41280004128120080172UQT1000032000에 속해있다.


In [ ]:
# space_plan
for i in parking_info.index:
    for j in space_plan.index:
        if parking_info.geometry[i].within(space_plan.geometry[j]):
            space_plan['라벨'][j] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
# culphy_plan
for i in parking_info.index:
    for j in culphy_plan.index:
        if parking_info.geometry[i].within(culphy_plan.geometry[j]):
            print('{}는 {}에 속해있다.'.format(parking_info.Name[i],culphy_plan.DGM_AR[j]))

고양제2_31는 957에 속해있다.
대화(임시)_25는 928에 속해있다.
덕양구청 부설주차장_309는 19832에 속해있다.
동구청 부설주차장_274는 24241에 속해있다.
마두(임시)_25는 774에 속해있다.
백석(임시)_27는 749에 속해있다.
일산서구청 부설주차장_366는 14160에 속해있다.
정발산(임시)_27는 820에 속해있다.


In [ ]:
# culphy_plan
for i in parking_info.index:
    for j in culphy_plan.index:
        if parking_info.geometry[i].within(culphy_plan.geometry[j]):
            culphy_plan['라벨'][j] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
# traffic_plan
for i in parking_info.index:
    for j in traffic_plan.index:
        if parking_info.geometry[i].within(traffic_plan.geometry[j]):
            print('{}는 {}에 속해있다.'.format(parking_info.Name[i],traffic_plan.DGM_AR[j]))

고양제1_94는 3815에 속해있다.
백마역_31는 999567에 속해있다.
백석1_247는 5426에 속해있다.
백석2_166는 5175에 속해있다.
백석환승터미널_300는 27028에 속해있다.
삼송 환승_185는 265230에 속해있다.
삼송 환승_185는 8926에 속해있다.
서정 제1_30는 919에 속해있다.
서정 제2_29는 961에 속해있다.
성사제1_22는 1009에 속해있다.
성사제2_33는 1370에 속해있다.
일산역_27는 999567에 속해있다.
일산제1_20는 663에 속해있다.
장항제1_193는 4644에 속해있다.
장항제2_195는 4626에 속해있다.
장항제3_215는 4974에 속해있다.
장항제4_214는 4980에 속해있다.
주교제1_359는 16310에 속해있다.
중산제1_9는 357에 속해있다.
중산제2_10는 549에 속해있다.
중산제3_32는 1111에 속해있다.
탄현5(공영)_57는 2150에 속해있다.
탄현제1_50는 1641에 속해있다.
탄현제2_39는 1406에 속해있다.
탄현제3_79는 2364에 속해있다.
탄현제4_47는 1368에 속해있다.
풍동제1_57는 1475에 속해있다.
풍동제2_76는 2397에 속해있다.
풍동제3_59는 2397에 속해있다.
행신 제4(임시)_111는 1471에 속해있다.
행신제1_40는 1369에 속해있다.
행신제2_44는 1375에 속해있다.
행신제3_43는 1468에 속해있다.
행신제4_44는 1471에 속해있다.
행신제5_49는 1462에 속해있다.
행신제6_46는 1463에 속해있다.
행신제7_24는 956에 속해있다.
행신제9_48는 1774에 속해있다.
행주산성 부설주차장 하_165는 55413에 속해있다.
행주산성 부설주차장 하_165는 55415에 속해있다.
화정환승_253는 3696에 속해있다.
신도공영_15는 593에 속해있다.


In [ ]:
# traffic_plan
for i in parking_info.index:
    for j in traffic_plan.index:
        if parking_info.geometry[i].within(traffic_plan.geometry[j]):
            traffic_plan['라벨'][j] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


### theat_muse 데이터 확인
- space_plan
- culphy_plan
- traffic_plan

In [ ]:
# space_plan
for i in theat_muse.index:
    for j in space_plan.index:
        if theat_muse.geometry[i].within(space_plan.geometry[j]):
            print('{}는 {}에 속해있다.'.format(theat_muse.Name[i],space_plan.MNUM[j]))

토당청소년수련관_공연장는 41280004128120080172UQT2200025000에 속해있다.


In [ ]:
# space_plan
for i in theat_muse.index:
    for j in space_plan.index:
        if theat_muse.geometry[i].within(space_plan.geometry[j]):
             space_plan['라벨'][j] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
# culphy_plan
for i in theat_muse.index:
    for j in culphy_plan.index:
        if theat_muse.geometry[i].within(culphy_plan.geometry[j]):
            print('{}는 {}에 속해있다.'.format(theat_muse.Name[i],culphy_plan.DGM_AR[j]))

테마동물원Zoozoo_박물관/미술관는 29711에 속해있다.
고양어린이박물관_박물관/미술관는 16463에 속해있다.
민속전시관_박물관/미술관는 4267에 속해있다.
아람미술관_박물관/미술관는 10665에 속해있다.
재단법인 중남미 문화원 병설 박물관_박물관/미술관는 13156에 속해있다.


In [ ]:
# culphy_plan
for i in theat_muse.index:
    for j in culphy_plan.index:
        if theat_muse.geometry[i].within(culphy_plan.geometry[j]):
            culphy_plan['라벨'][j] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
# traffic_plan
for i in theat_muse.index:
    for j in traffic_plan.index:
        if theat_muse.geometry[i].within(traffic_plan.geometry[j]):
            print('{}는 {}에 속해있다.'.format(theat_muse.Name[i],traffic_plan.DGM_AR[j]))

## nan값인 라벨 데이터에 5를 붙여준다.(계획도시 데이터에 계획되어 있다.)

In [ ]:
# nan값에 0을 붙여준다.
space_plan['라벨'].fillna(5,inplace=True)

In [ ]:
# centroid를 통해 중앙 자표값을 생성한다.
space_plan['geometry_Point'] = space_plan.geometry.centroid
space_plan = space_plan[['MNUM','ALIAS','REMARK','geometry','geometry_Point','분류','UQT_m','라벨']]
space_plan['geometry_Point'] = space_plan[['geometry_Point']].astype('str')
space_plan['X'] = space_plan.geometry_Point.str.split(' ').str[1]
space_plan['Y'] = space_plan.geometry_Point.str.split(' ').str[2]
space_plan['X'] = space_plan['X'].apply(lambda x : x[1:])
space_plan['Y'] = space_plan['Y'].apply(lambda x : x[:-1])
space_plan = space_plan[['MNUM','ALIAS','REMARK','geometry','X','Y','분류','UQT_m','라벨']]
space_plan

In [ ]:
real_space_plan = space_plan[space_plan['라벨'] !=1]
real_space_plan

In [ ]:
# nan값에 0을 붙여준다.
culphy_plan['라벨'].fillna(5,inplace=True)

In [ ]:
# centroid를 통해 중앙 자표값을 생성한다.
culphy_plan['geometry_Point'] = culphy_plan.geometry.centroid
culphy_plan = culphy_plan[['DGM_NM','DGM_AR','DGM_LT','geometry','geometry_Point','분류','라벨']]
culphy_plan['geometry_Point'] = culphy_plan[['geometry_Point']].astype('str')
culphy_plan['X'] = culphy_plan.geometry_Point.str.split(' ').str[1]
culphy_plan['Y'] = culphy_plan.geometry_Point.str.split(' ').str[2]
culphy_plan['X'] = culphy_plan['X'].apply(lambda x : x[1:])
culphy_plan['Y'] = culphy_plan['Y'].apply(lambda x : x[:-1])
culphy_plan = culphy_plan[['DGM_NM','DGM_AR','DGM_LT','geometry','X','Y','분류','라벨']]
culphy_plan 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  


,DGM_NM,DGM_AR,DGM_LT,geometry,X,Y,분류,라벨
0,기타 사회복지시설,2627,211,"MULTIPOLYGON (((126.86736 37.62940, 126.86761 ...",126.8678035135257,37.62941609459419,도시계획(공공문화체육시설),5.0
1,초등학교,13070,461,"MULTIPOLYGON (((126.90542 37.71515, 126.90619 ...",126.9062467951463,37.71512129388354,도시계획(공공문화체육시설),5.0
2,기타 사회복지시설,560,95,"MULTIPOLYGON (((126.78826 37.67892, 126.78836 ...",126.788205704093,37.67878561342879,도시계획(공공문화체육시설),1.0
3,초등학교,12000,444,"MULTIPOLYGON (((126.81611 37.68054, 126.81659 ...",126.8169627691664,37.68077824880825,도시계획(공공문화체육시설),5.0
4,유치원,3000,219,"MULTIPOLYGON (((126.81012 37.68488, 126.81062 ...",126.8105259693208,37.6847532762954,도시계획(공공문화체육시설),5.0
...,...,...,...,...,...,...,...,...
403,초등학교,6313,361,"MULTIPOLYGON (((126.78123 37.68526, 126.78126 ...",126.7816456342438,37.68489681428844,도시계획(공공문화체육시설),1.0
404,골프장,295598,3041,"MULTIPOLYGON (((126.85537 37.63687, 126.85545 ...",126.8584076302691,37.63594634957705,도시계획(공공문화체육시설),1.0
405,기타 사회복지시설,1063,137,"MULTIPOLYGON (((126.82721 37.70204, 126.82740 ...",126.8271365097641,37.70187565255021,도시계획(공공문화체육시설),1.0
406,기타 사회복지시설,935,150,"MULTIPOLYGON (((126.90365 37.70301, 126.90365 ...",126.9037482922718,37.70334943969106,도시계획(공공문화체육시설),1.0


In [ ]:
culphy_plan

,DGM_NM,DGM_AR,DGM_LT,geometry,X,Y,분류,라벨
0,기타 사회복지시설,2627,211,"MULTIPOLYGON (((126.86736 37.62940, 126.86761 ...",126.8678035135257,37.62941609459419,도시계획(공공문화체육시설),5.0
1,초등학교,13070,461,"MULTIPOLYGON (((126.90542 37.71515, 126.90619 ...",126.9062467951463,37.71512129388354,도시계획(공공문화체육시설),5.0
2,기타 사회복지시설,560,95,"MULTIPOLYGON (((126.78826 37.67892, 126.78836 ...",126.788205704093,37.67878561342879,도시계획(공공문화체육시설),1.0
3,초등학교,12000,444,"MULTIPOLYGON (((126.81611 37.68054, 126.81659 ...",126.8169627691664,37.68077824880825,도시계획(공공문화체육시설),5.0
4,유치원,3000,219,"MULTIPOLYGON (((126.81012 37.68488, 126.81062 ...",126.8105259693208,37.6847532762954,도시계획(공공문화체육시설),5.0
...,...,...,...,...,...,...,...,...
403,초등학교,6313,361,"MULTIPOLYGON (((126.78123 37.68526, 126.78126 ...",126.7816456342438,37.68489681428844,도시계획(공공문화체육시설),1.0
404,골프장,295598,3041,"MULTIPOLYGON (((126.85537 37.63687, 126.85545 ...",126.8584076302691,37.63594634957705,도시계획(공공문화체육시설),1.0
405,기타 사회복지시설,1063,137,"MULTIPOLYGON (((126.82721 37.70204, 126.82740 ...",126.8271365097641,37.70187565255021,도시계획(공공문화체육시설),1.0
406,기타 사회복지시설,935,150,"MULTIPOLYGON (((126.90365 37.70301, 126.90365 ...",126.9037482922718,37.70334943969106,도시계획(공공문화체육시설),1.0


In [ ]:
real_culphy_plan = culphy_plan[culphy_plan['라벨']!=1]
real_culphy_plan

,DGM_NM,DGM_AR,DGM_LT,geometry,X,Y,분류,라벨
0,기타 사회복지시설,2627,211,"MULTIPOLYGON (((126.86736 37.62940, 126.86761 ...",126.8678035135257,37.62941609459419,도시계획(공공문화체육시설),5.0
1,초등학교,13070,461,"MULTIPOLYGON (((126.90542 37.71515, 126.90619 ...",126.9062467951463,37.71512129388354,도시계획(공공문화체육시설),5.0
3,초등학교,12000,444,"MULTIPOLYGON (((126.81611 37.68054, 126.81659 ...",126.8169627691664,37.68077824880825,도시계획(공공문화체육시설),5.0
4,유치원,3000,219,"MULTIPOLYGON (((126.81012 37.68488, 126.81062 ...",126.8105259693208,37.6847532762954,도시계획(공공문화체육시설),5.0
6,기타 체육시설,609,132,"MULTIPOLYGON (((126.88355 37.66002, 126.88362 ...",126.8837653320626,37.66027104225559,도시계획(공공문화체육시설),5.0
7,기타 체육시설,300,83,"MULTIPOLYGON (((126.88064 37.66606, 126.88026 ...",126.880421496007,37.66609405198965,도시계획(공공문화체육시설),5.0
8,기타 체육시설,1039,171,"MULTIPOLYGON (((126.88052 37.66767, 126.88054 ...",126.8807899650519,37.66785315913823,도시계획(공공문화체육시설),5.0
11,중학교,9492,396,"MULTIPOLYGON (((126.81706 37.68020, 126.81711 ...",126.8179323322565,37.68016817688699,도시계획(공공문화체육시설),5.0
14,기타 사회복지시설,1029,184,"MULTIPOLYGON (((126.76178 37.69444, 126.76214 ...",126.7619656424406,37.6942741987259,도시계획(공공문화체육시설),5.0
16,기타 사회복지시설,1193,135,"MULTIPOLYGON (((126.89220 37.65219, 126.89230 ...",126.8924849467024,37.65214942287692,도시계획(공공문화체육시설),5.0


In [ ]:
# nan값에 0을 붙여준다.
traffic_plan['라벨'].fillna(5,inplace=True)

In [ ]:
# centroid를 통해 중앙 자표값을 생성한다.
traffic_plan['geometry_Point'] = traffic_plan.geometry.centroid
traffic_plan = traffic_plan[['DGM_NM','DGM_AR','DGM_LT','geometry','geometry_Point','분류','라벨']]
traffic_plan['geometry_Point'] = traffic_plan[['geometry_Point']].astype('str')
traffic_plan['X'] = traffic_plan.geometry_Point.str.split(' ').str[1]
traffic_plan['Y'] = traffic_plan.geometry_Point.str.split(' ').str[2]
traffic_plan['X'] = traffic_plan['X'].apply(lambda x : x[1:])
traffic_plan['Y'] = traffic_plan['Y'].apply(lambda x : x[:-1])
traffic_plan = traffic_plan[['DGM_NM','DGM_AR','DGM_LT','geometry','X','Y','분류','라벨']]
traffic_plan

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  


,DGM_NM,DGM_AR,DGM_LT,geometry,X,Y,분류,라벨
0,노외주차장,1930,269,"MULTIPOLYGON (((126.89527 37.70315, 126.89514 ...",126.8954978421802,37.70329162391477,도시계획(교통시설),5.0
1,노외주차장,1006,130,"MULTIPOLYGON (((126.79379 37.65890, 126.79391 ...",126.7936448658783,37.65872228216595,도시계획(교통시설),5.0
2,노외주차장,1111,134,"MULTIPOLYGON (((126.91072 37.71903, 126.91076 ...",126.9109444273574,37.71915380939122,도시계획(교통시설),5.0
3,노외주차장,631,104,"MULTIPOLYGON (((126.89455 37.62056, 126.89480 ...",126.8945860462858,37.62042561449137,도시계획(교통시설),5.0
4,노외주차장,987,132,"MULTIPOLYGON (((126.87241 37.59324, 126.87251 ...",126.8723205242999,37.59312860567663,도시계획(교통시설),5.0
...,...,...,...,...,...,...,...,...
318,노외주차장,2057,188,"MULTIPOLYGON (((126.80513 37.62288, 126.80527 ...",126.8050366071987,37.62267313718874,도시계획(교통시설),5.0
319,노외주차장,420,93,"MULTIPOLYGON (((126.85073 37.64267, 126.85075 ...",126.8505723936058,37.64269599696729,도시계획(교통시설),5.0
320,노외주차장,982,150,"MULTIPOLYGON (((126.86762 37.61254, 126.86787 ...",126.867826953065,37.61275405466562,도시계획(교통시설),5.0
321,노외주차장,621,106,"MULTIPOLYGON (((126.81503 37.60231, 126.81503 ...",126.8149524001323,37.6022239615025,도시계획(교통시설),1.0


In [ ]:
traffic_plan[traffic_plan['라벨']!=1]

,DGM_NM,DGM_AR,DGM_LT,geometry,X,Y,분류,라벨
0,노외주차장,1930,269,"MULTIPOLYGON (((126.89527 37.70315, 126.89514 ...",126.8954978421802,37.70329162391477,도시계획(교통시설),5.0
1,노외주차장,1006,130,"MULTIPOLYGON (((126.79379 37.65890, 126.79391 ...",126.7936448658783,37.65872228216595,도시계획(교통시설),5.0
2,노외주차장,1111,134,"MULTIPOLYGON (((126.91072 37.71903, 126.91076 ...",126.9109444273574,37.71915380939122,도시계획(교통시설),5.0
3,노외주차장,631,104,"MULTIPOLYGON (((126.89455 37.62056, 126.89480 ...",126.8945860462858,37.62042561449137,도시계획(교통시설),5.0
4,노외주차장,987,132,"MULTIPOLYGON (((126.87241 37.59324, 126.87251 ...",126.8723205242999,37.59312860567663,도시계획(교통시설),5.0
...,...,...,...,...,...,...,...,...
316,노외주차장,586,119,"MULTIPOLYGON (((126.81477 37.66771, 126.81479 ...",126.8146257896368,37.66755100834353,도시계획(교통시설),5.0
318,노외주차장,2057,188,"MULTIPOLYGON (((126.80513 37.62288, 126.80527 ...",126.8050366071987,37.62267313718874,도시계획(교통시설),5.0
319,노외주차장,420,93,"MULTIPOLYGON (((126.85073 37.64267, 126.85075 ...",126.8505723936058,37.64269599696729,도시계획(교통시설),5.0
320,노외주차장,982,150,"MULTIPOLYGON (((126.86762 37.61254, 126.86787 ...",126.867826953065,37.61275405466562,도시계획(교통시설),5.0
